<a href="https://colab.research.google.com/github/Shanjed/-NON-PRIMITIVE/blob/main/Project_Model_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="font-size: 14pt;">Prof. Krzysztof Rybinski</div><br/><br/>
<div style="font-size: 22pt;"><b>Artificial Intelligence course</b></div><br/><br/>
<div style="font-size: 18pt;">LAB 10</div><br/>
<div style="font-size: 18pt;">- RMP dataset</div><br/>
<div style="font-size: 18pt;">- emotion classification</div><br/>
<div style="font-size: 18pt;">- zero-shot classification</div><br/>

In [1]:
import torch

In [2]:
#exetute this cell only in Google Colab
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.4 MB/s eta 0:00:00


In [3]:
from transformers import pipeline
import os
import torch
import numpy as np
import pandas as pd
import pickle

In [4]:
#exetute this cell only in Google Colab
from google.colab import files
import io

In [5]:
#get index for the maximum score from prediction in model distilbert-base-uncased-emotion
############################################################
def get_max_index(ld):
    lv = [d['score'] for d in ld]
    lvind = lv.index(max(lv))
    return lvind
############################################################

In [6]:
#get label with the maximum score for model distilbert-base-uncased-mnli
############################################################
def get_max_label(ld):
    val = ld['scores']
    val_ind = lv.index(max(lv))
    label = ld['labels'][val_ind]
    return label
############################################################

In [7]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [8]:
#exetute this cell only in Google Colab
uploaded = files.upload()
rmp = pd.read_csv(io.BytesIO(uploaded['rmp_small.csv']))

Saving rmp_small.csv to rmp_small.csv


In [9]:
rmp

,review,score
0,"he can be a mean man sometimes, but just get o...",2
1,"He's fun, but he speeds through lectures and l...",2
2,Dr. Foster is an amazing teacher. One of my fa...,4
3,She is one of the greatest teacher I have ever...,4
4,Professor Bird was very easy and relaxed. If y...,3
...,...,...
97540,Charismatic and enthusiastic but that's it . H...,0
97541,"Great books - Dickens, Eliot, etc. A little d...",3
97542,Awesome professor. Doesnt stop explaining unti...,4
97543,"He has a really high standard, but grades fair...",4


In [10]:
sentences = list(rmp['review'])
labels = list(rmp['score'])

In [11]:
#for the final project skip this line
sentences = sentences[0:50000]
labels = labels[0:50000]

In [12]:
#do zero-shot classification
#check documentation https://huggingface.co/typeform/distilbert-base-uncased-mnli
classifier = pipeline("zero-shot-classification", 
                      model = "cross-encoder/nli-roberta-base",
                      device = 0 
)

In [13]:
l1 = "effective engagement of teachers with students"
l2 = "organization of the course and the study materials"
l3 = "professional teaching or skills of the professoe"
l4 = "Difficulty of the students understanding the professors"
l5 = "course outcome was beneficial or no"
l6 = "student teacher communication and helpfulness of teachers"
candidate_labels=[l1, l2, l3, l4, l5, l6]

In [14]:
len(sentences)

50000

In [15]:
prediction = classifier(sentences, candidate_labels=candidate_labels)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:646: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f84f215b700> was reported to be 50000 (when accessing len(dataloader)), but 297501 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:646: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f84f215b700> was reported to be 50000 (when accessing len(dataloader)), but 297502 samples have been fetched. 
  warnings.warn(warn_msg)
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:646: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7f84f215b700> was reported to be 50000 (when accessing len(dataloader)), but 297503 samples have been fetched. 
  warnings.warn(wa

In [16]:
prediction[90], prediction[101]

({'sequence': 'Frustrating. Most of the work was filler, and there wasn\'t much substance to the class. Some people call it "weeding-out" uninterested folk, I call it lack of organization.',
  'labels': ['Difficulty of the students understanding the professors',
   'professional teaching or skills of the professoe',
   'course outcome was beneficial or no',
   'student teacher communication and helpfulness of teachers',
   'organization of the course and the study materials',
   'effective engagement of teachers with students'],
  'scores': [0.33827710151672363,
   0.2805631458759308,
   0.1285964995622635,
   0.11205758154392242,
   0.08231500536203384,
   0.05819064378738403]},
 {'sequence': "Such a boring class, incredibly easy. If you're at all interested in economics don't take, very disappointed. Ended up with an A, yet didn't learn anything. The entire course is geared to spewing conservative trickle down economics facts. Someone could essentially miss every class and if you wen

In [17]:
lv = prediction[90]['scores']
lv

[0.33827710151672363,
 0.2805631458759308,
 0.1285964995622635,
 0.11205758154392242,
 0.08231500536203384,
 0.05819064378738403]

In [18]:
lvind = lv.index(max(lv))
lvind

0

In [19]:
l_max_labels = [get_max_label(val) for val in prediction]

In [20]:
df = pd.DataFrame(list(zip(l_max_labels, labels)), columns =['prof_opinion', 'score'])
df

,prof_opinion,score
0,professional teaching or skills of the professoe,2
1,professional teaching or skills of the professoe,2
2,student teacher communication and helpfulness ...,4
3,effective engagement of teachers with students,4
4,professional teaching or skills of the professoe,3
...,...,...
49995,professional teaching or skills of the professoe,4
49996,organization of the course and the study mater...,1
49997,professional teaching or skills of the professoe,2
49998,Difficulty of the students understanding the p...,0


In [21]:
#calculate crosstabulations
result = pd.crosstab(df["prof_opinion"], df["score"])
result

score,0,1,2,3,4
prof_opinion,,,,,
Difficulty of the students understanding the professors,2053,779,327,144,83
course outcome was beneficial or no,2758,1722,1658,1807,1995
effective engagement of teachers with students,48,145,518,1776,2687
organization of the course and the study materials,531,565,627,773,581
professional teaching or skills of the professoe,2560,1758,2464,5551,10004
student teacher communication and helpfulness of teachers,243,314,653,1951,2925


# Final project

1. Propose new classification, each group proposes its own </br>
2. Choose two zero shot models from https://huggingface.co/models?pipeline_tag=zero-shot-classification&language=en&sort=downloads </br>
3. Calculate predictions for each model on Google Colab</br>
4. Calculate cross-tabulation for each prediction </br>
5. Compare results, are there any differences </br>
6. Write conclusions about difference, speed of computation etc.</br>
7. Write what the results tell us about the student opinions regarding courses and professors </br>
8. Are there any recommendations for universities' management </br>
9. Make a presentation and send together with Jupyter Notbook to rybinski@rybinski.eu by Wednesday, 12 April 2023, 23:59
<br/><br/>
<div style="font-size: 16pt;">Grading rules</div><br/>

1. Project done in student teams <br/>
2. You can get max 4 points <br/>
3. 1 point for the classification and its justification <br/>
4. 1 point for the correct code for each model
5. 1 point for the summary, lessons learned and presentation clarity <br/>


In [ ]:
classifier = pipeline("zero-shot-classification", 
                      model = "facebook/bart-large-mnli",
                      device = 0
)

In [ ]:
predition = classifier(sentences, candidate_labels=candidate_labels)

In [ ]:
#modify if necessary
l_max_scores = [get_max_index(val) for val in prediction]

In [ ]:
#modify if necessary
#convert scores to labels
aaa = l_max_scores
aaa = ["poor or bad professor teaching skills" if x == 0 else x for x in aaa]
aaa = ["unfair or improper grading" if x == 1 else x for x in aaa]
aaa = ["poor or bad professor interpersonal skills or qualities" if x == 2 else x for x in aaa]
aaa = ["extremely or very difficult tests or exams" if x == 3 else x for x in aaa]
aaa = ["harassment or abuse or improper behavior" if x == 4 else x for x in aaa]
aaa = ["course or professor review or opinion" if x == 5 else x for x in aaa]
aaa[0:10]

In [ ]:
df = pd.DataFrame(list(zip(aaa, labels)), columns =['prof_opinion', 'score'])
df

In [ ]:
#calculate crosstabulations
result = pd.crosstab(df["prof_opinion"], df["score"])
result